<a href="https://colab.research.google.com/github/ClickerDG/ml_dropout/blob/main/ml_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Завантажуємо інструменти та бібліотеки, Keras та TensorFlow

In [ ]:
import tensorflow as tf
from tensorflow import keras

Завантажуємо дата-сет FashionMNIST та поділяємо дані на тестові, тренувальні та валідаційні

In [ ]:
(train_images, train_labels),(test_images, test_labels) = keras.datasets.fashion_mnist.load_data()
size = 500
epochs_size = 1000
train_labels = train_labels[:size]
test_labels = test_labels[:size]
train_images = train_images[:size] /  255.0
test_images = test_images[:size] / 255.0
validation_images = train_images[:size]
validation_labels = train_labels[:size]

Створюємо свою модель без шару Dropout

In [ ]:
class CustomModel(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.input_layer = keras.layers.Flatten(input_shape=(28,28))
        self.hidden1 = keras.layers.Dense(200, activation='relu')
        self.hidden2 = keras.layers.Dense(100, activation='relu')
        self.hidden3 = keras.layers.Dense(60, activation='relu')
        self.output_layer = keras.layers.Dense(10, activation='softmax')
    
    def call(self, input, training=None):
        input_layer = self.input_layer(input)
        hidden1 = self.hidden1(input_layer)
        hidden2 = self.hidden2(hidden1)
        hidden3 = self.hidden3(hidden2)
        output_layer = self.output_layer(hidden3)
        return output_layer

Завантажуємо розроблену модель та ініціалізуємо оптимізатори та гіперпараметри

In [ ]:
model = CustomModel()
sgd = keras.optimizers.SGD(lr=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Тренуємо модель (60 поколінь)

In [ ]:
model.fit(train_images, train_labels, epochs=epochs_size, validation_data=(validation_images, validation_labels), batch_size=size)

Epoch 1/1000
1/1 [==============================] - 0s 476ms/step - loss: 2.3510 - accuracy: 0.1300 - val_loss: 2.3371 - val_accuracy: 0.1300
Epoch 2/1000
1/1 [==============================] - 0s 54ms/step - loss: 2.3371 - accuracy: 0.1300 - val_loss: 2.3242 - val_accuracy: 0.1320
Epoch 3/1000
1/1 [==============================] - 0s 41ms/step - loss: 2.3242 - accuracy: 0.1320 - val_loss: 2.3122 - val_accuracy: 0.1380
Epoch 4/1000
1/1 [==============================] - 0s 48ms/step - loss: 2.3122 - accuracy: 0.1380 - val_loss: 2.3009 - val_accuracy: 0.1440
Epoch 5/1000
1/1 [==============================] - 0s 48ms/step - loss: 2.3009 - accuracy: 0.1440 - val_loss: 2.2902 - val_accuracy: 0.1460
Epoch 6/1000
1/1 [==============================] - 0s 41ms/step - loss: 2.2902 - accuracy: 0.1460 - val_loss: 2.2800 - val_accuracy: 0.1580
Epoch 7/1000
1/1 [==============================] - 0s 49ms/step - loss: 2.2800 - accuracy: 0.1580 - val_loss: 2.2703 - val_accuracy: 0.1720
Epoch 8/1000

Протестуємо на тестовому дата-сеті

In [ ]:
model.evaluate(test_images, test_labels)

16/16 [==============================] - 0s 2ms/step - loss: 0.6013 - accuracy: 0.7660


[0.6013422608375549, 0.765999972820282]

Створюємо свою модель за шаром Dropout

In [ ]:
class CustomModelDO(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.input_layer = keras.layers.Flatten(input_shape=(28,28))
        self.hidden1 = keras.layers.Dense(200, activation='relu')
        self.hidden2 = keras.layers.Dense(100, activation='relu')
        self.hidden3 = keras.layers.Dense(60, activation='relu')
        self.output_layer = keras.layers.Dense(10, activation='softmax')
        self.dropout_layer = keras.layers.Dropout(rate=0.2)
    
    def call(self, input, training=None):
        input_layer = self.input_layer(input)
        input_layer = self.dropout_layer(input_layer)
        hidden1 = self.hidden1(input_layer)
        hidden1 = self.dropout_layer(hidden1, training=training)
        hidden2 = self.hidden2(hidden1)
        hidden2 = self.dropout_layer(hidden2, training=training)
        hidden3 = self.hidden3(hidden2)
        hidden3 = self.dropout_layer(hidden3, training=training)
        output_layer = self.output_layer(hidden3)
        return output_layer

Завантажуємо розроблену модель та ініціалізуємо оптимізатори та гіперпараметри

In [ ]:
modelDO = CustomModelDO()
modelDO.compile(loss="sparse_categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

Тренуємо модель (60 поколінь)

In [ ]:
modelDO.fit(train_images, train_labels, epochs=epochs_size, validation_data=(validation_images, validation_labels), batch_size=size)

Epoch 1/1000
1/1 [==============================] - 1s 581ms/step - loss: 2.5107 - accuracy: 0.0880 - val_loss: 2.4016 - val_accuracy: 0.0620
Epoch 2/1000
1/1 [==============================] - 0s 50ms/step - loss: 2.4735 - accuracy: 0.1040 - val_loss: 2.3817 - val_accuracy: 0.0720
Epoch 3/1000
1/1 [==============================] - 0s 58ms/step - loss: 2.4649 - accuracy: 0.0760 - val_loss: 2.3635 - val_accuracy: 0.0780
Epoch 4/1000
1/1 [==============================] - 0s 47ms/step - loss: 2.4211 - accuracy: 0.1060 - val_loss: 2.3485 - val_accuracy: 0.0920
Epoch 5/1000
1/1 [==============================] - 0s 50ms/step - loss: 2.4368 - accuracy: 0.0980 - val_loss: 2.3352 - val_accuracy: 0.1080
Epoch 6/1000
1/1 [==============================] - 0s 45ms/step - loss: 2.4255 - accuracy: 0.0900 - val_loss: 2.3225 - val_accuracy: 0.1200
Epoch 7/1000
1/1 [==============================] - 0s 49ms/step - loss: 2.4168 - accuracy: 0.1160 - val_loss: 2.3103 - val_accuracy: 0.1280
Epoch 8/1000

Протестуємо на тестовому дата-сеті

In [ ]:
modelDO.evaluate(test_images, test_labels)

16/16 [==============================] - 0s 3ms/step - loss: 0.6581 - accuracy: 0.7520


[0.6581205129623413, 0.7519999742507935]